In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
# 주중-일자ver 파일
df_st = pd.read_csv('data/20230531_SUBWAY_MONTH_주중_일자ver.csv',encoding='utf-8', sep=',')
df_st['지하철역'] = df_st['지하철역'].str.replace('(', ' ',regex=False,).str.split().str[0]
for i in df_st.index:
    if df_st.loc[i, '지하철역'][-1] == '역':    # 서울역 -> 서울
        df_st.loc[i, '지하철역'] = df_st.loc[i, '지하철역'][:-1]
now = datetime.now()
f_date = now.strftime('%Y%m%d')
df_st.to_csv(f'data/{f_date}_SUBWAY_MONTH_주중_일자ver.csv',index=False,encoding='utf-8') # 0601자 파일생성

In [4]:
df = pd.read_csv(f'data/{f_date}_SUBWAY_MONTH_주중_일자ver.csv')
df.head()

,사용일자,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수,주중주말
0,20210101,일산선,백석,2724,2825,5549,주중
1,20210101,일산선,마두,1638,1652,3290,주중
2,20210101,3호선,을지로3가,1024,1009,2033,주중
3,20210101,3호선,종로3가,1255,1632,2887,주중
4,20210101,3호선,안국,2934,3029,5963,주중


In [5]:
df.호선명.nunique(), df.호선명.unique()

(25,
 array(['일산선', '3호선', '장항선', '중앙선', '4호선', '2호선', '1호선', '5호선', '6호선',
        '7호선', '수인선', '8호선', '9호선', '9호선2~3단계', '경강선', '경부선', '경원선', '경의선',
        '경인선', '경춘선', '공항철도 1호선', '과천선', '분당선', '안산선', '우이신설선'],
       dtype=object))

In [6]:
df.지하철역.nunique()

523

In [8]:
# 호선명 리스트로 추출해서 사용 
lines = df.호선명.unique().tolist()
df_dict = {line: df[df['호선명'] == line].copy() for line in lines}
for line, frame in df_dict.items():
    frame = df[df['호선명']==line].copy()   # 어떤 역할?
    frame = frame[['사용일자', '호선명', '지하철역', '승차총승객수', '하차총승객수', '승하차총승객수']]
    frame.loc[(frame['호선명'] == '2호선') & (frame['지하철역'] == '신천'), '지하철역'] = '잠실새내'
    frame.to_csv(f'data/main//temp_files/주중/{line}.csv',index=False,encoding='utf-8')

In [ ]:
############### 수정중

In [13]:
####### 호선 merge (파일 merge)########

# 1호선:서울지하철1호선(청량리~서울역), 경원선(소요산~청량리), 경부선(서울~천안/서동탄/광명), 장항선(천안~신창), 경인선(구로~인천)
# (공항철도 )
# 2호선, 5호선, 6호선, 7호선, 8호선
# 3호선: 서울지하철3호선(지축~오금), 일산선(대화~지축)
# 4호선: 서울지하철4호선(당고개~남태령), 과천선(남태령~금정), 안산선(금정~오이도)
# 9호선: 9호선, 9호선(2~3)단계
# 수인분당선: 수인선(인천~수원), 분당선()
# 경의중앙선: 경의선, 중앙선
# 우이신설선
path = 'data/main/temp_files/주중/'
line_info = [
    ([f'{path}1호선.csv', f'{path}공항철도 1호선.csv', f'{path}경부선.csv', f'{path}경원선.csv', f'{path}경인선.csv', f'{path}장항선.csv'], '1호선'),
    ([f'{path}2호선.csv'], '2호선'),
    ([f'{path}3호선.csv', f'{path}일산선.csv'], '3호선'),
    ([f'{path}4호선.csv', f'{path}과천선.csv', f'{path}안산선.csv'], '4호선'),
    ([f'{path}5호선.csv'], '5호선'),
    ([f'{path}6호선.csv'], '6호선'),
    ([f'{path}7호선.csv'], '7호선'),
    ([f'{path}8호선.csv'], '8호선'),
    ([f'{path}9호선.csv', f'{path}9호선2~3단계.csv'], '9호선'),
    ([f'{path}수인선.csv', f'{path}분당선.csv'], '수인분당선'),
    ([f'{path}경의선.csv', f'{path}중앙선.csv'], '경의중앙선'),
    ([f'{path}우이신설선.csv'], '우이신설선'),
    ([f'{path}경춘선.csv'], '경춘선'),
    ([f'{path}경강선.csv'], '경강선')
]
line_list = []
for df_list, line_name in line_info:
    df_copies = []
    for file in df_list:
        df = pd.read_csv(file)
        df_copies.append(df.copy())
    result = pd.concat(df_copies, axis=0)
    result = result.reset_index(drop=True)
    result.호선명 = line_name
    cols = list(result.columns)[:3]
    target = list(result.columns)[3:]
    res = result.groupby(cols)[target].agg('sum').reset_index()
    line_list.append(res)
    res.to_csv(f'data/main//lines/주중/{line_name}_일자.csv', index=False)
final = pd.concat(line_list,axis=0)


In [14]:
final.to_csv('data/main//lines/주중/merged_lines_일자.csv',index=False)   # // 두개의미?

```주말 파일들도 똑같은 식 수행```

In [15]:
# 주말-일ver 파일
df_st = pd.read_csv('data/20230531_SUBWAY_MONTH_주말_일자ver.csv',encoding='utf-8', sep=',')
df_st['지하철역'] = df_st['지하철역'].str.replace('(', ' ',regex=False,).str.split().str[0]
for i in df_st.index:
    if df_st.loc[i, '지하철역'][-1] == '역':    # 서울역 -> 서울
        df_st.loc[i, '지하철역'] = df_st.loc[i, '지하철역'][:-1]
now = datetime.now()
f_date = now.strftime('%Y%m%d')
df_st.to_csv(f'data/{f_date}_SUBWAY_MONTH_주말_일자ver.csv',index=False,encoding='utf-8') # 0601자 파일생성

In [16]:
df = pd.read_csv(f'data/{f_date}_SUBWAY_MONTH_주말_일자ver.csv')
df.head()

,사용일자,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수,주중주말
0,20210102,9호선,노들,1719,1527,3246,주말
1,20210102,우이신설선,삼양사거리,1112,1102,2214,주말
2,20210102,우이신설선,삼양,917,1019,1936,주말
3,20210102,우이신설선,화계,1470,1315,2785,주말
4,20210102,우이신설선,가오리,1653,1625,3278,주말


In [17]:
df.호선명.nunique(), df.호선명.unique()

(25,
 array(['9호선', '우이신설선', '안산선', '중앙선', '장항선', '일산선', '경인선', '수인선', '분당선',
        '과천선', '공항철도 1호선', '경춘선', '경의선', '경원선', '경부선', '경강선', '9호선2~3단계',
        '8호선', '7호선', '6호선', '5호선', '4호선', '3호선', '2호선', '1호선'],
       dtype=object))

In [18]:
df.지하철역.nunique()

523

In [19]:
# 호선명 리스트로 추출해서 사용 
lines = df.호선명.unique().tolist()
df_dict = {line: df[df['호선명'] == line].copy() for line in lines}
for line, frame in df_dict.items():
    frame = df[df['호선명']==line].copy()   # 어떤 역할?
    frame = frame[['사용일자', '호선명', '지하철역', '승차총승객수', '하차총승객수', '승하차총승객수']]
    frame.loc[(frame['호선명'] == '2호선') & (frame['지하철역'] == '신천'), '지하철역'] = '잠실새내'
    frame.to_csv(f'data/main//temp_files/주말/{line}.csv',index=False,encoding='utf-8')

In [21]:
####### 호선 merge (파일 merge)########

# 1호선:서울지하철1호선(청량리~서울역), 경원선(소요산~청량리), 경부선(서울~천안/서동탄/광명), 장항선(천안~신창), 경인선(구로~인천)
# (공항철도 )
# 2호선, 5호선, 6호선, 7호선, 8호선
# 3호선: 서울지하철3호선(지축~오금), 일산선(대화~지축)
# 4호선: 서울지하철4호선(당고개~남태령), 과천선(남태령~금정), 안산선(금정~오이도)
# 9호선: 9호선, 9호선(2~3)단계
# 수인분당선: 수인선(인천~수원), 분당선()
# 경의중앙선: 경의선, 중앙선
# 우이신설선
path = 'data/main/temp_files/주말/일자별/'
line_info = [
    ([f'{path}1호선.csv', f'{path}공항철도 1호선.csv', f'{path}경부선.csv', f'{path}경원선.csv', f'{path}경인선.csv', f'{path}장항선.csv'], '1호선'),
    ([f'{path}2호선.csv'], '2호선'),
    ([f'{path}3호선.csv', f'{path}일산선.csv'], '3호선'),
    ([f'{path}4호선.csv', f'{path}과천선.csv', f'{path}안산선.csv'], '4호선'),
    ([f'{path}5호선.csv'], '5호선'),
    ([f'{path}6호선.csv'], '6호선'),
    ([f'{path}7호선.csv'], '7호선'),
    ([f'{path}8호선.csv'], '8호선'),
    ([f'{path}9호선.csv', f'{path}9호선2~3단계.csv'], '9호선'),
    ([f'{path}수인선.csv', f'{path}분당선.csv'], '수인분당선'),
    ([f'{path}경의선.csv', f'{path}중앙선.csv'], '경의중앙선'),
    ([f'{path}우이신설선.csv'], '우이신설선'),
    ([f'{path}경춘선.csv'], '경춘선'),
    ([f'{path}경강선.csv'], '경강선')
]
line_list = []
for df_list, line_name in line_info:
    df_copies = []
    for file in df_list:
        df = pd.read_csv(file)
        df_copies.append(df.copy())
    result = pd.concat(df_copies, axis=0)
    result = result.reset_index(drop=True)
    result.호선명 = line_name
    cols = list(result.columns)[:3]
    target = list(result.columns)[3:]
    res = result.groupby(cols)[target].agg('sum').reset_index()
    line_list.append(res)
    res.to_csv(f'data/main//lines/주말/{line_name}_일자.csv', index=False)
final = pd.concat(line_list,axis=0)


In [22]:
final.to_csv('data/main//lines/주말/merged_lines_일자.csv',index=False)   # // 두개의미?